In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt

In [6]:
# Perform query and get results in JSON format via HTTP from local GeoSPARQL endpoint (GeoSPARQL Fuseki)

q1 = SPARQLWrapper("http://localhost:3031/spo")
q1.setQuery("""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX spo: <http://ulusalharita.tr/ont/spo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf:	<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT *
WHERE {

    ?osm rdf:type spo:OSM ; geo:hasGeometry ?osmg ; rdf:type ?osmt .
    ?osmt rdfs:subClassOf* spo:BuildingAndFacilities .
    ?osmg geo:asWKT ?osmwkt .
    

}
""")
q1.setReturnFormat(JSON)

r1 = q1.query().convert()

In [7]:
# Convert individuals and their results to a GeoDataFrame

ind = []
geo = []
typ = []

for r in r1["results"]["bindings"]:
    ind.append(r['osm']['value'].split('#')[1])
    geo.append(r['osmwkt']['value'])
    typ.append(r['osmt']['value'].split('#')[1])
    
t1 = pd.DataFrame({'individual':ind,'geometry':geo,'type':typ})
t1.drop_duplicates(inplace=True, ignore_index=True)
t1 = gpd.GeoDataFrame(t1, geometry=t1['geometry'].apply(wkt.loads))
#t1 = t1.set_crs('EPSG:5254')

In [8]:
ls_remove = ['AgriculturalAndLivestockFacility', 'EducationalFacility', 'EnergyFacility', 'HolySite', 'IndustrialFacility', 'MilitaryFacility',
'SocialFacility', 'SportsArea', 'StorageFacility', 'UrbanStructuresAndBuildings', 'Workplace']

rslt_df = t1.loc[t1['type'].isin(ls_remove)]

In [9]:
rslt_df = rslt_df.astype({'geometry': 'str'})

In [10]:
count = rslt_df.groupby(['individual','geometry'])['type'].count().reset_index().rename(columns={'type':'count'})

In [11]:
agg = rslt_df.groupby(['individual'])['type'].apply(','.join).reset_index().rename(columns={'individual':'ind'})

In [12]:
agg['typ2'] = agg['type'].str.split(',').apply(sorted).apply(lambda x: ','.join(map(str, x)))

AttributeError: Can only use .str accessor with string values!

In [13]:
rosm = pd.concat([count, agg], axis=1)

In [14]:
rosm = rosm[['individual','count','type','typ2','geometry']]

KeyError: "['typ2'] not in index"

In [261]:
rosm = gpd.GeoDataFrame(rosm, geometry=rosm['geometry'].apply(wkt.loads))

In [262]:
rosm.to_file('ndata/select/results_osm.gpkg', driver='GPKG')

## TOPO

In [15]:
# Perform query and get results in JSON format via HTTP from local GeoSPARQL endpoint (GeoSPARQL Fuseki)

q2 = SPARQLWrapper("http://localhost:3031/spo")
q2.setQuery("""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX spo: <http://ulusalharita.tr/ont/spo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf:	<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT *
WHERE {

    ?top rdf:type spo:TOPO1K ; geo:hasGeometry ?topg ; rdf:type ?topt .
    ?topt rdfs:subClassOf* spo:BuildingAndFacilities .
    ?topg geo:asWKT ?topwkt .
    

}
""")
q2.setReturnFormat(JSON)

r2 = q2.query().convert()

In [16]:
# Convert individuals and their results to a GeoDataFrame

ind = []
geo = []
typ = []

for r in r2["results"]["bindings"]:
    ind.append(r['top']['value'].split('#')[1])
    geo.append(r['topwkt']['value'])
    typ.append(r['topt']['value'].split('#')[1])
    
t2 = pd.DataFrame({'individual':ind,'type':typ ,'geometry':geo})
t2.drop_duplicates(inplace=True, ignore_index=True)
t2 = gpd.GeoDataFrame(t2, geometry=t2['geometry'].apply(wkt.loads))
t2 = t2.set_crs('EPSG:4326')

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)

In [46]:
ls_remove_t = ['AgriculturalAndLivestockFacility', 'EducationalFacility', 'EnergyFacility', 'HolySite', 'IndustrialFacility', 'MilitaryFacility',
'SocialFacility', 'SportsArea', 'StorageFacility', 'UrbanStructuresAndBuildings', 'Workplace']

rslt_df_t = t2.loc[t2['type'].isin(ls_remove_t)]

In [47]:
rslt_df_t = rslt_df_t.astype({'geometry': 'str'})

In [48]:
count_t = rslt_df_t.groupby(['individual','geometry'])['type'].count().reset_index().rename(columns={'type':'count'})

In [49]:
agg_t = rslt_df_t.groupby(['individual'])['type'].apply(','.join).reset_index().rename(columns={'individual':'ind'})

In [50]:
agg_t['typ2'] = agg_t['type'].str.split(',').apply(sorted).apply(lambda x: ','.join(map(str, x)))

In [51]:
rtopo = pd.concat([count_t, agg_t], axis=1)

In [52]:
rtopo = rtopo[['individual','count','type','typ2','geometry']]

In [53]:
rtopo = gpd.GeoDataFrame(rtopo, geometry=rtopo['geometry'].apply(wkt.loads))

In [59]:
rtopo = rtopo.set_crs('EPSG:4326')
rtopo

,individual,count,type,typ2,geometry
0,topo10050,1,HolySite,HolySite,"POLYGON ((28.97220 41.01709, 28.97227 41.01713..."
1,topo10070,1,Workplace,Workplace,"POLYGON ((28.94316 41.01909, 28.94310 41.01905..."
2,topo10080,1,Workplace,Workplace,"POLYGON ((28.94624 41.02295, 28.94629 41.02290..."
3,topo10119,1,Workplace,Workplace,"POLYGON ((28.97107 41.01424, 28.97106 41.01425..."
4,topo10156,1,UrbanStructuresAndBuildings,UrbanStructuresAndBuildings,"POLYGON ((28.98587 41.01388, 28.98585 41.01384..."
...,...,...,...,...,...
1001,topo9893,1,Workplace,Workplace,"POLYGON ((28.95437 41.01882, 28.95446 41.01886..."
1002,topo9904,1,Workplace,Workplace,"POLYGON ((28.97517 41.01311, 28.97513 41.01313..."
1003,topo9941,1,Workplace,Workplace,"POLYGON ((28.94316 41.02943, 28.94316 41.02943..."
1004,topo9943,1,Workplace,Workplace,"POLYGON ((28.97069 41.00551, 28.97046 41.00556..."


In [1]:
# Plot results on a more sophisticated web map

import folium
from folium.plugins import MiniMap
from folium.plugins import MeasureControl
from folium.features import GeoJsonTooltip, GeoJsonPopup

m = folium.Map(location=[41.011015,28.968244],tiles='cartodbpositron',zoom_start=14)


green = lambda x: {'fillColor': '#00ff00',
                   'color': '#00ff00',
                   'opacity': 0.8}

red = lambda x: {'fillColor': '#ff0000',
                   'color': '#ff0000',
                   'opacity': 0.8}

tooltip=GeoJsonTooltip(
    fields=["individual", 'typ2', 'count',],
    aliases=['ID:','Sınıf:',"Sınıf Sayısı:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """
)

folium.GeoJson(rtopo,
               name='Buildings',
               style_function=red,
               show=True,
               tooltip=tooltip,
               ).add_to(m)


folium.LayerControl().add_to(m)
MiniMap(tile_layer='cartodbpositron',zoom_level_offset=-6, toggle_display=True).add_to(m)
MeasureControl().add_to(m)
m

NameError: name 'rtopo' is not defined

In [287]:
rtopo.to_file('ndata/select/results_topo.gpkg', driver='GPKG')

In [31]:
# Perform query and get results in JSON format via HTTP from local GeoSPARQL endpoint (GeoSPARQL Fuseki)

q1 = SPARQLWrapper("http://localhost:3031/spo")
q1.setQuery("""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX spo: <http://ulusalharita.tr/ont/spo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf:	<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT *
WHERE {

    ?osm rdf:type spo:OSM ; geo:hasGeometry ?osmg ; rdf:type ?osmt .
    ?osmt rdfs:subClassOf* spo:MixedType .
    ?osmg geo:asWKT ?osmwkt .
    
OPTIONAL{?osm spo:hasName ?osmn}
}
""")
q1.setReturnFormat(JSON)

r1 = q1.query().convert()

In [40]:
# Convert individuals and their results to a GeoDataFrame

ind = []
geo = []
typ = []
nam = []

for r in r1["results"]["bindings"]:
    ind.append(r['osm']['value'].split('#')[1])
    geo.append(r['osmwkt']['value'])
    typ.append(r['osmt']['value'].split('#')[1])
    
for r in r1["results"]["bindings"]:
    try:
        nam.append(r['osmn']['value'])
    except:
        nam.append('Isimsiz Yer')
    
t1 = pd.DataFrame({'individual':ind,'type':typ,'name':nam,'geometry':geo})
t1.drop_duplicates(inplace=True, ignore_index=True)
t1 = gpd.GeoDataFrame(t1, geometry=t1['geometry'].apply(wkt.loads))
Buildings = t1.set_crs('EPSG:4326')

In [42]:
Buildings

,individual,type,name,geometry
0,osm13020,MixedType,Isimsiz Yer,"POLYGON ((28.94754 41.01939, 28.94747 41.01935..."
1,osm4708,MixedType,Isimsiz Yer,"POLYGON ((28.94235 41.02103, 28.94247 41.02093..."
2,osm4752,MixedType,Isimsiz Yer,"POLYGON ((28.94254 41.02131, 28.94241 41.02132..."
3,osm781,MixedType,Isimsiz Yer,"POLYGON ((28.97774 41.00850, 28.97765 41.00858..."
4,osm4717,MixedType,Isimsiz Yer,"POLYGON ((28.94230 41.02108, 28.94212 41.02097..."
5,osm13247,MixedType,Isimsiz Yer,"POLYGON ((28.94625 41.01984, 28.94636 41.01990..."
6,osm7090,MixedType,Isimsiz Yer,"POLYGON ((28.94292 41.02081, 28.94287 41.02085..."
7,osm13026,MixedType,Isimsiz Yer,"POLYGON ((28.94752 41.01922, 28.94768 41.01931..."
8,osm813,MixedType,Isimsiz Yer,"POLYGON ((28.94314 41.02138, 28.94313 41.02131..."
9,osm1210,MixedType,Isimsiz Yer,"POLYGON ((28.97552 41.00882, 28.97548 41.00885..."


In [43]:
# Plot results on a more sophisticated web map

import folium
from folium.plugins import MiniMap
from folium.plugins import MeasureControl
from folium.features import GeoJsonTooltip, GeoJsonPopup

m = folium.Map(location=[41.011015,28.968244],tiles='cartodbpositron',zoom_start=14)


green = lambda x: {'fillColor': '#00ff00',
                   'color': '#00ff00',
                   'opacity': 0.8}

tooltip=GeoJsonTooltip(
    fields=["individual", 'name', 'type',],
    aliases=['ID:',"Detay Adı:",'Sınıf:'],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """
)

folium.GeoJson(Buildings,
               name='Buildings',
               style_function=green,
               show=True,
               tooltip=tooltip,
               ).add_to(m)


folium.LayerControl().add_to(m)
MiniMap(tile_layer='cartodbpositron',zoom_level_offset=-6, toggle_display=True).add_to(m)
MeasureControl().add_to(m)
m